In [1]:
import os
import random
from io import open
import unicodedata
import string
import re

import torch
import torchaudio
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from pathlib import Path
import kaldi_io
import math
import torch.utils.data as Data

from lib.Data_show import Data_show
from lib.Word_cla_Dataset import Word_cla_Dataset
from lib.Decoder import Decoder
from collections import Counter

%matplotlib inline


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 加载数据SLR85训练集

In [4]:
SLR_feats = { u:d for u,d in kaldi_io.read_mat_scp("/home1/meichaoyang/Dataset/feats/SLR85/hifi_16k/train/feats.scp") }
SLR_feats_dev = { u:d for u,d in kaldi_io.read_mat_scp("/home1/meichaoyang/Dataset/feats/SLR85/hifi_16k/dev/feats.scp") }

In [5]:
phone_label = { u:d for u,d in kaldi_io.read_vec_int_ark("feats/ali.1.ph") }

In [6]:
feats = { u:d for u,d in kaldi_io.read_mat_scp("../../git/phone_align_more_44k/train_fbank/feats.scp") }

In [ ]:
 "你"      "好"       "米"       "雅"
129 63    61 27     128 64     92 69

In [7]:
word =np.array([[128,62],[60,26], [127, 63], [91,68] ])

#### 生成word_label

In [8]:
word_label={}
for utt, label in phone_label.items():
    word_label_tmp = np.zeros(label.shape[0]) 
    i = 0
    while i < label.shape[0]:
        finded = False
        if label[i] in word[:,0]:
            word_index = np.where(word == label[i])[0][0]
            start = i
            while i < label.shape[0]:
                
                if label[i] == label[start]:
                    i += 1
                elif label[i] == word[word_index][1]:
                    finded = True
                    end = i
                    i += 1
                else:
                    i -= 1
                    break
            
            if finded:
                word_label_tmp[start:end+1] = word_index+1
                        
                        
            
        i += 1
        
    word_label[utt] = word_label_tmp.astype(np.int16)

In [11]:
phone_label["SV0255_7_01_S3881"]

array([  1,   1, 128, 128, 128,  62,  62,  62,  60,  60,  60,  60,  60,
        26,  26,  26,  26,  26,  26,  26,  26,  26,  26,   1,   1,   1,
         1, 127, 127, 127,  63,  63,  63,  63,  91,  91,  68,  68,  68,
        68,  68,  68,   1,   1,   1,   1])

In [12]:
word_label["SV0255_7_01_S3881"]

array([0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0,
       0, 0], dtype=int16)

In [10]:
feats_new={}
for key,values in feats.items():
    if "SV" not in key:
        if random.random() < 0.4:
            feats_new[key] = values
feat_train = {**feats_new, **SLR_feats}

In [12]:
len(feats_new)

35449

## 自定义数据集

In [9]:
class Data_show:
    class2word=[]
    
    def __init__(self, class2word =["other", "你","好","米","雅"]):
        
        Data_show.class2word = class2word


    def show_softmax(self, pred_label_with_softmax):
        
        title = ""
        for i in range(len(Data_show.class2word)):
            title += "\t" + Data_show.class2word[i]
        
        content = ""
        for i in range(0,pred_label_with_softmax.shape[0]):
            content += str(i) + ":\t"
            for j in range(0,pred_label_with_softmax.shape[1]):
                content += '%.2f\t' %pred_label_with_softmax[i][j]
            content += "\n"
     
        return title, content

In [13]:
feat_new["IC0104W0039"][367-20:367+10].reshape(1,-1).shape

NameError: name 'feat_new' is not defined

In [14]:
data_set_train = Word_cla_Dataset(word_label, feat_train)

In [15]:
i = 0
j = 0

for key in word_label.keys():
    if "SV" in key:
        i += 1
    else :
        j += 1
print(i ,j)

23627 89330


In [16]:
data_set_dev = Word_cla_Dataset(word_label, SLR_feats_dev)

In [15]:
data_set_train.word_label_nd[-75:-61]

array([0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3])

In [17]:
print(data_set_train.feats_nd.shape)
print(data_set_train.word_label_nd.shape)
print(data_set_train.word_label_nd[1440])

(13814620, 1200)
(13814620,)
0


In [19]:
data_set_train[-1]

[array([[13.633668, 13.99829 , 15.901409, ..., 13.34208 , 14.012131,
         13.291359]], dtype=float32),
 array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4])]

In [20]:
data_set_train[100][0].shape

(1, 1200)

In [21]:
data_set_train[100][1].shape

(245,)

In [22]:
train_data = torch.Tensor(data_set_train.feats_nd)#.to(device)
train_label = torch.LongTensor(data_set_train.word_label_nd)#.to(device)
dev_data = torch.Tensor(data_set_dev.feats_nd)#.to(device)
dev_label = torch.LongTensor(data_set_dev.word_label_nd)#.to(device)

In [23]:
print('Counter(data)\n',Counter(data_set_train.word_label_nd))

Counter(data)
 Counter({0: 11761262, 4: 707501, 2: 646999, 3: 444466, 1: 407874})


# 模型搭建

In [24]:
class DNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, num_classes)


    def forward(self, input):
        x = torch.sigmoid(self.fc1(input))
        x = torch.sigmoid(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        x = self.fc5(x)
        
        return x

## 训练迭代

In [25]:
LEARNING_RATE = 0.001  #0.001
EPOCH = 2      #400 best
BATCH_SIZE = 150
input_size=40*30
num_classes=5

In [26]:
print(train_data.shape)
print(train_label.shape)

torch.Size([13968102, 1200])
torch.Size([13968102])


In [ ]:
training_set = Data.TensorDataset(train_data,
                                  train_label)
training_loader = Data.DataLoader(dataset=training_set,
                                      batch_size=BATCH_SIZE,
                                      shuffle=True)
testing_set = Data.TensorDataset(dev_data,
                                 dev_label)
testing_loader = Data.DataLoader(dataset=testing_set,
                                     batch_size=BATCH_SIZE,
                                     shuffle=False)
model = DNN(input_size, num_classes)#.to(device)
# criterion = nn.CrossEntropyLoss(weight=torch.tensor([3.5,3.0,3.0,2.5,2.5,5,2.0,2.5,1.5,1.0]).to(device))
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0,14.0,10.0,13.5,10.5]))#.to(device))
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
for epoch in range(EPOCH):
    correct_train = 0
    total_train = 0
    
    total_loss = 0
    ite = 0
    print_every = 2000
    for (data, label) in training_loader:
        ite +=1
        data = data
        label = label
        pred_label = model(data)
        loss = criterion(pred_label, label)
        optim.zero_grad()
        loss.backward()
        total_loss += loss.item()
        optim.step()
        _, answer = torch.max(pred_label.data, 1)
        total_train += label.size(0)
        correct_train += (answer == label).sum()
        if ite % print_every == 0:
            print("total_loss:",total_loss/print_every, "\tloss:",loss.item())
            total_loss=0
    print('Epoch {:3d} Accuracy on training data: {}% ({}/{})'
          .format(epoch, (100 * correct_train / total_train), correct_train, total_train))
    # pytorch 0.4 feature, not calculate grad on test set
    with torch.no_grad():
        correct_test = 0
        total_test = 0
        for (data, label) in testing_loader:
            pred_label = model(data)
            _, answer = torch.max(pred_label.data, 1)
            total_test += label.size(0)
            correct_test += (answer == label).sum()
        print('          Accuracy on testing data: {}% ({}/{})'
              .format((100 * correct_test / total_test), correct_test, total_test))



total_loss: 1.5707216240167619 	loss: 1.581739068031311
total_loss: 1.5668513911366462 	loss: 1.5658981800079346
total_loss: 1.5659097527265549 	loss: 1.5520520210266113
total_loss: 1.5662817475795745 	loss: 1.5839787721633911
total_loss: 1.565831024825573 	loss: 1.567136526107788
total_loss: 1.5653219010233879 	loss: 1.5240556001663208
total_loss: 1.5636251400113106 	loss: 1.5908160209655762
total_loss: 1.5648815143704415 	loss: 1.6106810569763184
total_loss: 1.5648323717713355 	loss: 1.5643868446350098
total_loss: 1.5653951781988145 	loss: 1.6069931983947754
total_loss: 1.5655405352711678 	loss: 1.6075032949447632
total_loss: 1.5654067241549492 	loss: 1.5709507465362549
total_loss: 1.5655875853896142 	loss: 1.5863605737686157
total_loss: 1.5660399155020714 	loss: 1.6260517835617065
total_loss: 1.5643442229032516 	loss: 1.5317611694335938
total_loss: 1.5664346075057984 	loss: 1.524701476097107
total_loss: 1.5658028916716575 	loss: 1.5197653770446777
total_loss: 1.565809099316597 	loss

### 预测

In [ ]:
# model = torch.load('model.pkl')
# model.to(device)

In [36]:
test_feats = { u:d for u,d in kaldi_io.read_mat_scp("/home1/meichaoyang/workspace/align_44_1k/test_feat/feats.scp")}
test_feats.keys()

dict_keys(['miya_mcy', 'miya_mcy1', 'miya_mingzhang', 'miya_mingzhang1'])

In [37]:
feat = test_feats["miya_mcy1"]
feats_list = []
for i in range(20,feat.shape[0]-9):
    input_data=feat[i-20:i+10].reshape(1,-1)
    feats_list.append(input_data)
feats_nd = np.concatenate(tuple(feats_list))

In [381]:
i=30
input_feat = feat[i-20:i+10].reshape(1,-1)

In [52]:
# test_feats = { u:d for u,d in kaldi_io.read_mat_scp("/home1/meichaoyang/workspace/align_44_1k/test_feat/feats.scp")}

pred_label = model(torch.Tensor(feats_nd).to(device))
_, answer = torch.max(pred_label.data, 1)
answer_list=list(answer.to("cpu", torch.int).numpy())

In [73]:
utt='SV0296_7_01_S3895'
# utt="SV0255_7_01_S3881"
# utt=list(phone_label.keys())[110]

# label_list = list(phone_label_dev[utt])
pred_label = model(torch.Tensor(SLR_feats_dev[utt]).to(device))
_, answer = torch.max(pred_label.data, 1)
answer_list=list(answer.to("cpu", torch.int).numpy())

In [74]:
list(SLR_feats_dev.keys())[-1]

'SV0296_7_01_S3900'

In [75]:
# utt=list(feats_aishell.keys())[110]

# label_list = list(feats_aishell[utt])
# pred_label = model(torch.Tensor(feats_aishell[utt]))#.to(device)
# _, answer = torch.max(pred_label.data, 1)
# answer_list=list(answer.to("cpu", torch.int).numpy())

## 将预测结果映射

In [76]:
title, content = Data_show().show_softmax(torch.nn.Softmax()(pred_label))

/home1/meichaoyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [77]:
print(title,"\n",content)

	other	你	好	米	雅 
 0:	0.98	0.00	0.01	0.00	0.00	
1:	0.80	0.13	0.01	0.02	0.04	
2:	0.96	0.01	0.02	0.00	0.01	
3:	0.98	0.00	0.01	0.00	0.00	
4:	0.99	0.00	0.01	0.00	0.00	
5:	0.89	0.02	0.02	0.01	0.06	
6:	0.98	0.01	0.00	0.01	0.00	
7:	0.99	0.00	0.00	0.01	0.00	
8:	0.93	0.03	0.01	0.01	0.02	
9:	0.27	0.15	0.06	0.17	0.34	
10:	0.03	0.50	0.00	0.47	0.00	
11:	0.02	0.70	0.00	0.28	0.00	
12:	0.02	0.70	0.00	0.29	0.00	
13:	0.00	0.50	0.00	0.49	0.00	
14:	0.00	0.28	0.00	0.71	0.00	
15:	0.00	0.33	0.00	0.67	0.00	
16:	0.00	0.41	0.00	0.59	0.00	
17:	0.00	0.64	0.00	0.36	0.00	
18:	0.00	0.31	0.00	0.65	0.03	
19:	0.00	0.34	0.00	0.63	0.02	
20:	0.00	0.31	0.00	0.67	0.02	
21:	0.00	0.32	0.00	0.66	0.01	
22:	0.00	0.33	0.00	0.64	0.03	
23:	0.00	0.38	0.00	0.50	0.11	
24:	0.00	0.31	0.00	0.47	0.22	
25:	0.01	0.31	0.00	0.30	0.37	
26:	0.00	0.37	0.02	0.36	0.25	
27:	0.00	0.17	0.05	0.11	0.66	
28:	0.00	0.09	0.16	0.02	0.72	
29:	0.00	0.03	0.32	0.01	0.64	
30:	0.00	0.01	0.89	0.00	0.10	
31:	0.00	0.01	0.92	0.00	0.07	
32:	0.00	0.00	0.94	0.00	0.06	
33:

In [78]:
word_label[utt]

array([0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0], dtype=int16)

### “你好米雅”测试

In [42]:
feats_miya_test = { u:d for u,d in kaldi_io.read_mat_scp("/home1/meichaoyang/Dataset/feats/SLR85/far_field/train/feats.scp") }

In [43]:
utt_miya_test=list(feats_miya_test.keys())[420]
# utt_aishell="IC0001W0406"

pred_label_miya_test = model(torch.Tensor(feats_miya_test[utt_miya_test]).to(device))
_, answer_miya_test = torch.max(pred_label_miya_test.data, 1)
answer_miya_test_list=list(answer_miya_test.to("cpu", torch.int).numpy())

In [44]:
title, content = Data_show().show_softmax(torch.nn.Softmax()(pred_label_miya_test))

/home1/meichaoyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [45]:
print(title)
print(content)

	other	你	好	米	雅
0:	0.37	0.54	0.01	0.05	0.03	
1:	0.18	0.59	0.02	0.09	0.12	
2:	0.15	0.67	0.01	0.16	0.01	
3:	0.20	0.55	0.01	0.05	0.19	
4:	0.17	0.41	0.15	0.06	0.20	
5:	0.38	0.38	0.05	0.07	0.13	
6:	0.86	0.09	0.02	0.02	0.01	
7:	0.26	0.60	0.01	0.04	0.09	
8:	0.87	0.10	0.00	0.01	0.01	
9:	0.98	0.01	0.00	0.01	0.00	
10:	0.82	0.15	0.01	0.02	0.00	
11:	0.98	0.01	0.00	0.01	0.00	
12:	0.99	0.00	0.00	0.00	0.00	
13:	0.99	0.00	0.00	0.01	0.00	
14:	0.71	0.25	0.01	0.03	0.01	
15:	0.50	0.43	0.01	0.03	0.02	
16:	0.53	0.42	0.01	0.03	0.02	
17:	0.43	0.50	0.03	0.04	0.01	
18:	0.95	0.03	0.00	0.01	0.00	
19:	0.75	0.19	0.01	0.04	0.02	
20:	0.45	0.48	0.01	0.04	0.02	
21:	0.77	0.17	0.01	0.02	0.03	
22:	0.98	0.01	0.00	0.01	0.00	
23:	0.24	0.64	0.01	0.04	0.07	
24:	0.84	0.13	0.01	0.02	0.01	
25:	0.54	0.34	0.02	0.03	0.08	
26:	0.35	0.54	0.01	0.04	0.06	
27:	0.99	0.01	0.00	0.01	0.00	
28:	0.37	0.47	0.02	0.04	0.09	
29:	0.98	0.00	0.01	0.01	0.00	
30:	0.98	0.01	0.00	0.01	0.00	
31:	0.23	0.52	0.02	0.09	0.14	
32:	0.93	0.02	0.01	0.01	0.03	
33:	0

In [ ]:
feats[utt_miya_test]

## 非“你好米雅”测试

In [47]:
feats_aishell = { u:d for u,d in kaldi_io.read_mat_scp("../../wake_dnn_miya_only/feats_aishell2_test/feats.scp") }

In [48]:
utt_aishell=list(feats_aishell.keys())[420]
utt_aishell="IC0001W0406"

pred_label_aishell = model(torch.Tensor(feats_aishell[utt_aishell]).to(device))
_, answer_aishell = torch.max(pred_label_aishell.data, 1)
answer_aishell_list=list(answer_aishell.to("cpu", torch.int).numpy())

In [49]:
title, content = Data_show().show_softmax(torch.nn.Softmax()(pred_label_miya_test))

/home1/meichaoyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [50]:
print(title)
print(content)

	other	你	好	米	雅
0:	0.37	0.54	0.01	0.05	0.03	
1:	0.18	0.59	0.02	0.09	0.12	
2:	0.15	0.67	0.01	0.16	0.01	
3:	0.20	0.55	0.01	0.05	0.19	
4:	0.17	0.41	0.15	0.06	0.20	
5:	0.38	0.38	0.05	0.07	0.13	
6:	0.86	0.09	0.02	0.02	0.01	
7:	0.26	0.60	0.01	0.04	0.09	
8:	0.87	0.10	0.00	0.01	0.01	
9:	0.98	0.01	0.00	0.01	0.00	
10:	0.82	0.15	0.01	0.02	0.00	
11:	0.98	0.01	0.00	0.01	0.00	
12:	0.99	0.00	0.00	0.00	0.00	
13:	0.99	0.00	0.00	0.01	0.00	
14:	0.71	0.25	0.01	0.03	0.01	
15:	0.50	0.43	0.01	0.03	0.02	
16:	0.53	0.42	0.01	0.03	0.02	
17:	0.43	0.50	0.03	0.04	0.01	
18:	0.95	0.03	0.00	0.01	0.00	
19:	0.75	0.19	0.01	0.04	0.02	
20:	0.45	0.48	0.01	0.04	0.02	
21:	0.77	0.17	0.01	0.02	0.03	
22:	0.98	0.01	0.00	0.01	0.00	
23:	0.24	0.64	0.01	0.04	0.07	
24:	0.84	0.13	0.01	0.02	0.01	
25:	0.54	0.34	0.02	0.03	0.08	
26:	0.35	0.54	0.01	0.04	0.06	
27:	0.99	0.01	0.00	0.01	0.00	
28:	0.37	0.47	0.02	0.04	0.09	
29:	0.98	0.00	0.01	0.01	0.00	
30:	0.98	0.01	0.00	0.01	0.00	
31:	0.23	0.52	0.02	0.09	0.14	
32:	0.93	0.02	0.01	0.01	0.03	
33:	0

In [ ]:
utt_aishell

## 保存模型

In [ ]:
torch.save(model.to("cpu"), 'model.pkl')
model1 = torch.load('model.pkl')

In [ ]:
sm = torch.jit.script(model1)
sm.save("phone_cla_model.pt")

In [ ]:
model1 = torch.load('model.pkl')

In [ ]:
model1

In [ ]:
map1={1:"1-1",2:"2-1",3:"3-1"}

In [ ]:
map2={2:"2-2",3:"3-2",4:"4-2"}

In [ ]:
{**map1,**map2}

In [ ]:
{**map2,**map1}